In [ ]:
%%writefile flask_ex1.py

# import Flask
from flask import Flask

# initialize the Flask app, note that all routing blocks use @app
app = Flask(__name__)  # instantiate a flask app object

# routing blocks - note there is only one in this case - @app.route('/') 

# home page - the first place your app will go
@app.route('/', methods = ['GET', 'POST'])  # GET is the default, more about GET and POST below
# the function below will be executed at the host and port followed by '/' 
# the name of the function that will be executed at '/'. Its name is arbitrary.
def index():
    return 'Hello!'

# no more routing blocks

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)
    # the combination of host and port are where we look for it in our browser
    # e.g. http://0.0.0.0:8080/
    # setting debug=True to see error messages - wouldn't do this on a final deployed app



In [ ]:
%%writefile flask_ex2.py

from flask import Flask
app = Flask(__name__)

# first routing block
@app.route('/') # if no methods specified, default is 'GET'
def index():
    display_str = 'Hello!'
    # code below makes a button to go to the '/rainbow' block
    go_to_rainbow_html = '''
        <form action="/rainbow" >
            <input type="submit" value = "Go to rainbow"/>
        </form>
    '''
    # html that gets returned
    return display_str + go_to_rainbow_html


# second routing block
@app.route('/rainbow') # if no methods specified, default is 'GET'
def rainbow():
    colors = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
    color_divs = []
    for i in range(20):
        color = colors[i % len(colors)]
        # making html string that makes colors
        div = '''<div style="background-color: {0};
                             color: white;
                             text-align: center;">
                     {1}
                 </div>'''.format(color, '*' * 100)
        color_divs.append(div)
    
    # make html that gives us a button to go back to the home page
    go_to_home_html = '''
        <form action="/" >
            <input type="submit" value = "Go home"/>
        </form>
    '''
    # combine them for final html
    return '''
            <!DOCTYPE html>
            <html>
                <head>
                    <meta charset="utf-8">
                    <title>Rainbow</title>
                </head>
                <body>{0}</body>
            </html>
            '''.format('\n'.join(color_divs)) + go_to_home_html

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)


In [ ]:
%%writefile flask_ex3.py

from collections import Counter
from flask import Flask, request
app = Flask(__name__)

# formatting dictionary keys and values to be displayed in html
def dict_to_html(d):
    return '<br>'.join('{0}: {1}'.format(k, d[k]) for k in sorted(d))


# Form page to submit text
@app.route('/')
def submission_page():
    # in this form, method = 'POST' means that data entered into
    # the 'user_input' variable will be sent to the /word_counter routing
    # block when the 'Enter text' submit button is pressed
    return '''
        <form action="/word_counter" method='POST' >
            <input type="text" name="user_input" />
            <input type="submit" value = 'Enter text'/>
        </form>
        '''


# My word counter app
# this routing block accepts data in variable 'user_input' from the form
# and then processes it on the server and returns word counts to the browser
@app.route('/word_counter', methods=['GET', 'POST'])
def word_counter():
    text = str(request.form['user_input']) #gets the value entered in the input type="text", name="user_input"
    word_counts = Counter(text.lower().split())
    
    # formatted output string
    page = 'There are {0} words.<br><br>Individual word counts:<br> {1}'
    
    # make html that gives us a button to go back to the home page
    go_to_home_html = '''
        <form action="/" >
            <input type="submit" value = "Enter more text"/>
        </form>
    '''
    
    return page.format(len(word_counts), dict_to_html(word_counts)) + go_to_home_html


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)



In [ ]:
%%writefile flask_ex4.py

from random import random
import matplotlib
from matplotlib.figure import Figure
matplotlib.use('Agg')
from io import BytesIO
from flask import Flask
app = Flask(__name__)

@app.route('/')
def index():
    # img in the html specifies the presence of an image
    # however, the src of the image is a routing block instead of
    # a .jpg or .png
    return '''
        <h2>Here's my plot!</h2>
        <img src='/plot.png'>
        '''

@app.route('/plot.png')
def get_graph():
    fig = Figure()
    ax = fig.subplots()
    n = 10
    ax.plot(range(n), [random() for i in range(n)])
    image = BytesIO()
    fig.savefig(image)  #the plot is saved
    return image.getvalue(), 200, {'Content-Type': 'image/png'}


if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)



In [ ]:
%%writefile templates/table.html

<html>
<head>
<!-- CSS style -->
<style>
table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
}
th, td {
    padding: 15px;
}
table.center {
    margin-left:auto; 
    margin-right:auto;
}
</style>
</head>
<body>
<table class="center">
  <thead>
    <th>x</th>
    <th>y</th>
  </thead>
  <tbody>
    <!-- Here is where the data passed into the template gets inserted -->
    {% for x, y in data %}
      <tr>
        <td>{{ x }}</td>
        <td>{{ y }}</td>
      </tr>
    {% endfor %}
  </tbody>
</table>

In [ ]:
%%writefile flask_ex5.py

from flask import Flask, render_template
from random import random
app = Flask(__name__)


@app.route('/')
def index():
    n = 10
    x = range(n)
    y = [round(random(),3) for i in x]
    # table.html is an existing file in the templates folder
    # isn't this nice, no html here!
    return render_template('table.html', data=zip(x, y))  # using Jinja here, the render_template call

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)



In [ ]:
# Here we train and pickle a logistic regression model on the iris dataset

# Based on:
# http://scikit-learn.org/stable/auto_examples/linear_model/plot_iris_logistic.html

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
import pickle

# import data
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
Y = iris.target

# declare model
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')

# fit the data
logreg.fit(X, Y)

# pickle the data (wouldn't normally do this but want to plot prediction vs the train data)
trainXY = np.hstack((X,Y.reshape(-1,1)))
with open('data/trainXY.pkl', 'wb') as f:
    pickle.dump(trainXY, f)

# pickle the model
with open('data/model_logreg.pkl', 'wb') as f:
    pickle.dump(logreg, f)

In [ ]:
%%writefile flask_ex6.py

from flask import Flask, request
import pickle
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO

# Initialize app
app = Flask(__name__)

# load the pickled model
with open('data/model_logreg.pkl', 'rb') as f:
    model = pickle.load(f)

# load the pickled training data to display with prediction
with open('data/trainXY.pkl', 'rb') as f:
    trainXY = pickle.load(f)

trainX = trainXY[:,:2]
trainY = trainXY[:,2]

# Home page with form on it to submit new data
@app.route('/')
def get_new_data():
    return '''
        <form action="/predict" method='POST'>
          Sepal length (4.0 - 9.0):<br>
          <input type="text" name="length"> 
          <br>
          Sepal width (1.0 - 5.0):<br>
          <input type="text" name="width"> 
          <br><br>
          <input type="submit" value="Submit for class prediction">
        </form>
        '''

@app.route('/predict', methods = ["GET", "POST"])
def predict():
    # request the text from the form 
    length = float(request.form['length'])
    width = float(request.form['width'])
    X_n = np.array([[length, width]])
    
    # predict on the new data
    Y_pred = model.predict(X_n)

    # for plotting 
    X_0 = trainX[trainY == 0] # class 0
    X_1 = trainX[trainY == 1] # class 1
    X_2 = trainX[trainY == 2] # class 2
    
    # color-coding prediction 
    if Y_pred[0] == 0:
        cp = 'b'
    elif Y_pred[0] == 1:
        cp = 'r'
    else:
        cp = 'g'

    if plt:
        plt.clf() # clears the figure when browser back arrow used to enter new data

    plt.scatter(X_0[:, 0], X_0[:, 1], c='b', edgecolors='k', label = 'class 0')
    plt.scatter(X_1[:, 0], X_1[:, 1], c='r', edgecolors='k', label = 'class 1')
    plt.scatter(X_2[:, 0], X_2[:, 1], c='g', edgecolors='k', label = 'class 2')
    plt.scatter(X_n[:, 0], X_n[:, 1], c=cp, edgecolors='k', marker = 'd', \
        s=100, label = 'prediction')
    plt.xlabel('Sepal length')
    plt.ylabel('Sepal width')
    plt.title('Prediction plotted with training data')
    plt.legend()
        
    image = BytesIO()
    plt.savefig(image)
    out = image.getvalue(), 200, {'Content-Type': 'image/png'}
    return out

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080, debug=True)

In [4]:
from flask import Flask, render_template, request, jsonify
import pickle
from build_model import TextClassifier

app = Flask(__name__)


with open('static/model.pkl', 'rb') as f:
    model = pickle.load(f)


@app.route('/', methods=['GET'])
def index():
    """Render a simple splash page."""
    return render_template('index.html')

@app.route('/submit', methods=['GET'])
def submit():
    """Render a page containing a textarea input where the user can paste an
    article to be classified.  """
    return render_template('submit.html')

@app.route('/predict', methods=['POST'])
def predict():
    """Recieve the article to be classified from an input form and use the
    model to classify.
    """
    data = str(request.form['article_body'])
    pred = str(model.predict([data])[0])
    return render_template('predict.html', article=data, predicted=pred)


if __name__ == '__main__':
    app.run(host='0.0.0.0',port=8080, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

In [5]:
%tb

SystemExit: 1